<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pandas-datareader

In [0]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date


In [0]:
def get_data(symbol, start, end):
  # Use IEX API for stats so we get all info we need  IEX Cloud consol 
  https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,companyName,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
  # From here we can get the following information
  """
  symbol	string	
  companyName	string	Company name of the security
  marketcap	number	Market cap of the security calculated as shares outstanding * previous day close.
  week52high	number	
  week52low	number	
  week52change	number	Percentage change
  sharesOutstanding	number	Number of shares outstanding as the difference between issued shares and treasury shares. Investopedia
  avg30Volume	number	Average 30 day volume
  avg10Volume	number	Average 10 day volume
  float	number	Returns the annual shares outstanding minus closely held shares.
  employees	number	
  ttmEPS	number	Trailing twelve month earnings per share. Investopedia
  ttmDividendRate	number	Trailing twelve month dividend rate per share
  dividendYield	number	The ratio of trailing twelve month dividend compared to the previous day close price. The dividend yield is represented as a percentage calculated as (ttmDividendRate) / (previous day close price) Investopedia
  nextDividendDate	string	Expected ex date of the next dividend
  exDividendDate	string	Ex date of the last dividend
  nextEarningsDate	string	Expected next earnings report date
  peRatio	number	Price to earnings ratio calculated as (previous day close price) / (ttmEPS)
  beta	number	Beta is a measure used in fundamental analysis to determine the volatility of an asset or portfolio in relation to the overall market. Levered beta calculated with 1 year historical data and compared to SPY.
  day200MovingAvg	number	
  day50MovingAvg	number	
  maxChangePercent	number	
  year5ChangePercent	number	
  year2ChangePercent	number	
  year1ChangePercent	number	
  ytdChangePercent	number	
  month6ChangePercent	number	
  month3ChangePercent	number	
  month1ChangePercent	number	
  day30ChangePercent	number	
  day5ChangePercent	number	
  
  
  """
  
  
  data =  dr.DataReader(symbol, 'iex', start, end)[['close']]
  return data.rename(columns={'close': symbol})

SyntaxError: ignored

<h2>Authenticate User </h2>

In [0]:
from google.colab import auth
auth.authenticate_user()

<h3> Loading Nasdaq and Nyse shares </h3>

In [0]:
!gsutil cp gs://datascience-bucket-mm/nyse-companylist.csv /tmp/nyse.csv
!gsutil cp gs://datascience-bucket-mm/nasdaq-companylist.csv /tmp/nyse.csv  
  
# Print the result to make sure the transfer worked.
#!cat /tmp/nyse.csv

Copying gs://datascience-bucket-mm/nyse-companylist.csv...
/ [1 files][392.3 KiB/392.3 KiB]                                                
Operation completed over 1 objects/392.3 KiB.                                    
Copying gs://datascience-bucket-mm/nasdaq-companylist.csv...
/ [1 files][462.5 KiB/462.5 KiB]                                                
Operation completed over 1 objects/462.5 KiB.                                    


<h3>  Testing Historical API </h3>

In [0]:

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol=ticker)
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_historical_data(ticker, start, end):
  df = get_statistics(ticker)
  return df
  
  

def get_date_ranges():
  end_date = date.today()
  start_date = end_date - BDay(60)
  return start_date, end_date

def test():
  start,end = get_date_ranges()
  print('start:{}, end:{}'.format(start_date, end_date))
  print(get_historical_data('AMZN', start_date, end_date))
  
def get_latest_price(symbol):
  base_url = "https://cloud.iexapis.com/stable/stock/{ticker}/quote?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=symbol,close".format(ticker=symbol)
  import requests
  return pd.read_csv(base_url)

def get_latest_price_yahoo(symbol, as_of_date):
  return dr.get_data_yahoo(symbol, as_of_date, as_of_date)[['Close']]
  
    
  

<h3> Reading source data and computing performance </h3>

In [0]:
def get_nyse_df():
  return pd.read_csv('/tmp/nyse.csv', header=0)[['Symbol', 'Name', 'Sector', 'industry']]


In [0]:
def compute_performance(start_dt, end_dt, ticker):
  try:
    import time
    historical_df =  get_historical_data(ticker, start_dt, end_dt)
    latest_df = get_latest_price(ticker)
    merged = pd.merge(historical_df, latest_df, how='inner' , left_on="Symbol", right_on="symbol").drop('symbol', axis=1)
    return merged
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))

In [0]:
def find_best_performing(start_dt, end_dt):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  nyse_df = get_nyse_df()
  symbols = nyse_df['Symbol'].values.tolist()
  print('Now we have to source data for:{}'.format(len(symbols)))
  dfs = (compute_performance(start_dt, end_dt, symbol) for symbol in symbols)
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  return pd.merge(nyse_df, all_data, how='inner', on='Symbol' )[['Symbol', 'Name', 'Sector', 'industry', 'companyName','close', 
       'month1ChangePercent','month3ChangePercent', 'month6ChangePercent',  'day200MovingAvg', 'day50MovingAvg']]
  

start_dt, end_dt = get_date_ranges()
perf_df = find_best_performing(start_dt, end_dt)#compute_performance(start_dt, end_dt)
print(perf_df.columns)
print(perf_df.shape)

Finding Best Performing Stocks between:2019-05-01 00:00:00-2019-07-24
Now we have to source data for:3524
Exception:HTTP Error 404: Not Found
Unable to find data for ACCP:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for ZTEST:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for BBRX:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for BLIN          :HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for CRUSC:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for CNTX:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for DDOC:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for IGLE:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for EVGBC:HTTP Error 404: Not Found
Exception:HTTP Error 404: Not Found
Unable to find data for EV

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Index(['Symbol', 'Name', 'Sector', 'industry', 'companyName', 'close',
       'month1ChangePercent', 'month3ChangePercent', 'month6ChangePercent',
       'day200MovingAvg', 'day50MovingAvg'],
      dtype='object')
(3488, 11)


In [0]:
# Sorting 
perf_df.sort_values(by=['month1ChangePercent'], inplace=True, ascending=False)
perf_df.head(30)


,Symbol,Name,Sector,industry,companyName,close,month1ChangePercent,month3ChangePercent,month6ChangePercent,day200MovingAvg,day50MovingAvg
690,CTXRW,"Citius Pharmaceuticals, Inc.",Health Care,Major Pharmaceuticals,"Citius Pharmaceuticals, Inc.",0.3000,3.285714,0.578947,-0.285714,0.2786,0.1174
301,ASV,"ASV Holdings, Inc.",Capital Goods,Construction/Ag Equipment/Trucks,"ASV Holdings, Inc.",6.9600,2.962571,1.773800,1.208439,3.2500,3.7700
181,AMRHW,"Ameri Holdings, Inc.",Technology,Semiconductors,"Ameri Holdings, Inc.",0.0900,2.000000,2.000000,-0.400000,0.1657,0.0496
2466,SKIS,"Peak Resorts, Inc.",Consumer Services,Hotels/Resorts,"Peak Resorts, Inc.",10.8400,1.844156,1.433333,1.319031,4.7300,4.5600
3012,TRNX,"Taronis Technologies, Inc.",Technology,Industrial Machinery/Components,"Taronis Technologies, Inc.",0.5180,1.347347,-0.185764,-0.879744,2.4932,0.3582
1602,IFMK,iFresh Inc.,Consumer Services,Food Chains,"iFresh, Inc.",1.9400,1.166667,0.962264,0.507246,1.2400,1.2300
1063,EVOK,"Evoke Pharma, Inc.",Health Care,Major Pharmaceuticals,"Evoke Pharma, Inc.",1.1400,1.050847,0.689944,-0.572438,1.7800,0.7100
31,ACST,"Acasti Pharma, Inc.",Health Care,Major Pharmaceuticals,"Acasti Pharma, Inc.",2.3400,1.009901,1.255556,0.970874,0.9900,1.1000
1081,EYEGW,"Eyegate Pharmaceuticals, Inc.",Health Care,Major Pharmaceuticals,"EyeGate Pharmaceuticals, Inc.",0.0290,1.000000,-0.200000,-0.555556,0.0683,0.0400
604,CETXW,Cemtrex Inc.,Capital Goods,Industrial Machinery/Components,"Cemtrex, Inc.",0.0400,1.000000,-0.800000,-0.800000,0.1548,0.0276


<h3> Group by sector, to find best performers </h3>

In [0]:
res = perf_df[['industry', 'month1ChangePercent','month3ChangePercent', ]].groupby(['industry']).mean().sort_values(by=['month1ChangePercent','month1ChangePercent'], ascending=False)
res.head(20)

,month1ChangePercent,month3ChangePercent
industry,,
Construction/Ag Equipment/Trucks,0.623505,0.391324
Food Chains,0.224757,0.162845
Oil/Gas Transmission,0.196226,0.054032
Shoe Manufacturing,0.180125,-0.129228
Other Pharmaceuticals,0.177430,0.163765
Medical Electronics,0.145786,-0.042202
Aluminum,0.128936,-0.149153
Hotels/Resorts,0.112513,-0.001065
Auto Manufacturing,0.101077,0.450148


In [0]:
# Panda test
base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
df = pd.read_csv(base_url)
df['symbol'] = 'AMZN'
df.columns
  

Index(['companyName', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'symbol'],
      dtype='object')

In [0]:
get_statistics('AMZN')

,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent
0,"Amazon.com, Inc.",1.843297,1871.88,1744.88,0.230806,0.066052,0.121094


In [0]:
get_latest_price('AMZN')

,symbol,close
0,AMZN,2009.9


In [0]:
dr.get_data_yahoo('AMZN', date(2019,7,16), date(2019,7,16))


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-07-16,2026.319946,2001.219971,2010.579956,2009.900024,2553900,2009.900024


In [0]:
def get_historical_price(symbol, as_of_date):
  # Leveraging Historical
  import requests
  historical_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/chart/date/{cob}?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true'.format(
                        symbol=symbol, cob=as_of_date.strftime('%Y%m%d'))
  print('Fetching data for {} from:{}'.format(symbol, historical_url))
  return requests.get(historical_url).json()

get_historical_price('AMZN', date(2019,7,16))

Fetching data for AMZN from:https://cloud.iexapis.com/stable/stock/AMZN/chart/date/20190716?token=sk_98e397d4bee940488e1f48e9b419508f&chartByDay=true


[{'change': 0,
  'changeOverTime': 0,
  'changePercent': 0,
  'close': 2009.9,
  'date': '2019-07-16',
  'high': 2026.32,
  'label': 'Jul 16',
  'low': 2001.22,
  'open': 2010.58,
  'uClose': 2009.9,
  'uHigh': 2026.32,
  'uLow': 2001.22,
  'uOpen': 2010.58,
  'uVolume': 2618198,
  'volume': 2618198}]

In [0]:
def get_batch_request():
  import requests
  iex_url = 'https://cloud.iexapis.com/stable/stock/aapl/batch?token=sk_98e397d4bee940488e1f48e9b419508f&types=quote,stats&range=1d&last=1'
  return requests.get(iex_url).json()
get_batch_request()

{'quote': {'avgTotalVolume': 22770576,
  'calculationPrice': 'tops',
  'change': -1.15,
  'changePercent': -0.00562,
  'close': 204.5,
  'closeTime': 1563307200182,
  'companyName': 'Apple, Inc.',
  'delayedPrice': 203.99,
  'delayedPriceTime': 1563392746921,
  'extendedChange': 0.93,
  'extendedChangePercent': 0.00457,
  'extendedPrice': 204.28,
  'extendedPriceTime': 1563370200392,
  'high': 205.091,
  'iexAskPrice': 0,
  'iexAskSize': 0,
  'iexBidPrice': 0,
  'iexBidSize': 0,
  'iexLastUpdated': 1563393597008,
  'iexMarketPercent': 0.033215496439546736,
  'iexRealtimePrice': 203.35,
  'iexRealtimeSize': 20,
  'iexVolume': 436383,
  'lastTradeTime': 1563393597008,
  'latestPrice': 203.35,
  'latestSource': 'IEX real time price',
  'latestTime': '3:59:57 PM',
  'latestUpdate': 1563393597008,
  'latestVolume': 13137934,
  'low': 203.66,
  'marketCap': 935629618000,
  'open': 204.05,
  'openTime': 1563370200444,
  'peRatio': 16.96,
  'previousClose': 204.5,
  'previousVolume': None,
  '

In [0]:
start, end = get_date_ranges()
compute_performance(start, end, 'AMZN')
#get_latest_price('AMZN')

Historical ok:Index(['companyName', 'beta', 'day50MovingAvg', 'day200MovingAvg',
       'month6ChangePercent', 'month3ChangePercent', 'month1ChangePercent',
       'Symbol'],
      dtype='object')
latest ok:Index(['symbol', 'close'], dtype='object')


,companyName,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent,Symbol,close
0,"Amazon.com, Inc.",1.846598,1882.87,1747.01,0.187028,0.077798,0.065678,AMZN,1992.03


<h3> Testing Yahoo Historical </h3>

In [0]:
#testing VIX
from scipy.stats import pearsonr
import requests

def get_historical_value(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    return data.rename(columns={'Adj Close' : symbol})
    
  except Exception as e :
    print('Excepiton for {}:{}'.format(symbol, str(e)))
    return pd.DataFrame(columns=[symbol])

<h3> Getting VIX values </h3>

In [0]:
vix_vals = get_historical_value('^VIX')


<h3>Gettign all stocks from IEX API </h3>

In [0]:
# Get all Stocks from IEX
import requests
def get_all_stocks_data():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token=sk_98e397d4bee940488e1f48e9b419508f').json()
  good_ones = [d['symbol'] for d in all_symbols_data if d['isEnabled']]
  return map(lambda symbol: (symbol, get_historical_value(symbol)), good_ones)

In [57]:
def calculate_correlation(vix, all_stocks):
  result = []
  best = 0
  for symbol, vals in all_stocks:
    if vals.shape[0] == vix.shape[0]:
      concats  = pd.concat([vix, vals], axis = 1)
      corr_matrix = concats.corr(method='pearson')
      corr_with_vix = corr_matrix.loc['^VIX'][1]
      if corr_with_vix > 0 and corr_with_vix > best:
        print('New Corr with {}:{}'.format(symbol, corr_with_vix))
        best = corr_with_vix
  return best
      
  #return sorted(result, key = lambda tpl: tpl[1])
all_stocks_historical = get_all_stocks_data()
best = calculate_correlation(vix_vals, all_stocks_historical)                  
                  

New Corr with AAME:0.07980534120952935
Excepiton for ABEOW:'Date'
Excepiton for ABR-A:No data fetched for symbol ABR-A using YahooDailyReader
Excepiton for ABR-B:No data fetched for symbol ABR-B using YahooDailyReader
Excepiton for ABR-C:No data fetched for symbol ABR-C using YahooDailyReader
Excepiton for ACAMU:'Date'
Excepiton for ACAMW:'Date'
New Corr with ACIU:0.10587772550727555
Excepiton for ACTTW:'Date'
Excepiton for ADILW:'Date'
New Corr with AEMD:0.13749066947343994
Excepiton for AEP-B:No data fetched for symbol AEP-B using YahooDailyReader
Excepiton for AFGB:'Date'
Excepiton for AFHBL:'Date'
Excepiton for AGBAR:'Date'
Excepiton for AGBAU:'Date'
Excepiton for AGBAW:'Date'
Excepiton for AGFSW:'Date'
Excepiton for AGM-C:No data fetched for symbol AGM-C using YahooDailyReader
Excepiton for AGM-D:No data fetched for symbol AGM-D using YahooDailyReader
Excepiton for AGM.A:No data fetched for symbol AGM.A using YahooDailyReader
Excepiton for AGO-B:No data fetched for symbol AGO-B us

KeyboardInterrupt: ignored

<h3> Getting list of all ETFS </h3>



In [0]:
import pandas as pd
xl = pd.read_excel('https://topforeignstocks.com/wp-content/uploads/2019/02/Complete-List-of-Vanguard-ETFs-Feb-2019.xlsx')

In [60]:
xl.shape
xl.columns
  
  
  
  




Index(['S.No.', 'Name', 'Ticker'], dtype='object')

In [0]:
len(res)

NameError: ignored